# Reinforcement Learning Project

MULIUKOV Artem

NASCIMENTO Leandro


This project is an study of the Rainbow Reinforcement Learning Model, from the article: "Rainbow: Combining Improvements in Deep Reinforcement Learning", available at: https://arxiv.org/abs/1710.02298

Notice: this notebook is an adaptation of the notebook 08 from the repo: https://github.com/Curt-Park/rainbow-is-all-you-need

## The Model

The Rainboe model basically assemble 6 different DQN (Deep Q-Network) modifications in order to achieve an overall more robust RL agent, specially ehen tested with the Atari games environment. The 6 DQN modificatons chosen by the authors were: 

- DDQN (Double DQN)
- PER (Prioritized Experience Replay)
- Dueling Network
- Multi-step learning
- Distributional RL
- Noisy Networks

We will do an overview of each of these variations as well as of the integrated model (Rainbow).

### DDQN

As it is widely known that regular Q-learning has an over-optimism towards its estimation of the Q-values for the states (even with the $\epsilon$-greedy strategy), the DDQN tries to solve this issue by partially decoupling the maximization step of the Q-function. This transforms the the loss function of DQN, that was:
$$ \big(R_{t+1} + \gamma_{t+1} \max_{a'} Q_{\bar{\theta}}(S_{t+1},a') - Q_{\theta}(S_t, A_t) \big)^2 $$
to:

$$ (R_{t+1} + \gamma_{t+1} Q_{\bar{\theta}}(S_{t+1}, \arg\max_{a'} Q_{\theta}(S_{t+1}, a; \theta_t)) -  Q_{\theta}(S_t, A_t) )^2$$
This separation of selection of action from the evaluation reduces the general overestimations of DQN.


### Prioritized Replay

The paper shows that in many games, this was the factor that most improved performance on the Rainbow model. This is becauase it changes the way data is sampled from the replay buffer, a major parto of DQN. Originally, DQN samples uniformly at random from the buffer. PER changes this uniform distribution into a new $p_t$, proportional to:

$$p_t \propto |R_{t+1} + \gamma_{t+1} \max_{a'} Q_{\bar{\theta}}(S_{t+1},a') - Q_{\theta}(S_t, A_t)|^\omega  $$

$w$ being another hyper-parameter. This method creates a bias for choosing more recent transitions.

### Dueling networks 

This method is basically a decomposition of the Q-value function in a "value" and "advantage" stream, with an encoder for the state variable. This results in the following function:

$$ q_{\theta}(s,a) = v_\eta (f_\xi (s)) + a_\psi(f_\xi(s),a) - \frac{\sum_{a'}{a_\psi (f_\xi(s),a')}}{N_{\text{actions}}} $$

In this case, $\theta$ is the concatenation of $\xi,\eta,\psi$ 

### Multi-step learning 

The idea here is to make a forward view greater than one step for the reward, as it is normally done. So instead of using $R_{t+1}$, we define: $ R_t^n \equiv \sum_{k=0}^{n-1} \gamma_t^k R_{t+k+1} $. Also, we replace $\gamma_{t+1}$ in the loss function for:
$ \gamma_t^n = \prod_{i=1}^k \gamma_{t+i}$


### Distributional RL

This variation maybe is the one which changes DQN the most. DQN minimizes (the expactation) of the loss, since it is stochastic (or mini-batch). The idea is then try to instead find the distribution of returns instead of the expected return. This is done with probability masses. We have a vector $\mathbb{z}$, where each $z^i = v_{\min} + (i-1) \frac{v_{\max} - v_{\min}}{N_{\text{atoms}}-1}$
with $i \in \{1,...,N_{\text{atoms}}\}$.
Then, we have that the distribution of each atom $i$ at time t is $d_t = (\mathbb{z}, \mathbb{p}_\theta(S_t,A_t)$.

Finally, the new loss to be minimized will be the Kullback-Leiber divergence ($D_{KL}$) between $d_t$ and the target $d_t' \equiv R_{t+1} + \gamma_{t+1}\mathbf{z}, p_{\bar{\theta}}(S_{t+1},\bar{a}^*_{t+1})$, with $\bar{a}^*_{t+1} = \arg\max_a q_{\bar{\theta}}(S_{t+1},a) $ being the greedy action for the optimal policy.
More precisely, the $D_{KL}$ is calculated between the L2-projection of $d_t'$ and $d_t$, that is $D_{KL}(\mathbb{\Phi_z}d_t' || d_t)$.

### Noisy Networks

The premisse of Noisy Nets is that, for complex systems, the smallest difference on the weights of the network will make the agent too much state dependent agent after multiple time-steps. So, the idea is to add noise on the network in order to improve the agents exploration in the state space. This happens because the network will eventually ignore the noise, but at different rates and differents parts of the state space.

Given a linear network as $\mathbf{y} = \mathbf{Wx + b}$, we transform it into:
$$ \mathbf{y} = \mathbf{Wx + b + b_{noisy} }\odot \epsilon^b + (\mathbf{W}_{noisy} \odot \epsilon^w) \mathbf{x} $$
where the $\epsilon$ are factorised Gaussian noise random variables.

### The integrated agent
As the authors recommend, the first step is to change the distributional loss to a n-step version, such that $ d_t^n = (R_t^n + \gamma_t^n z, p_{\bar{\theta}}(S_{t+n}, \bar{a}^*_{t+n}))$

Then, a greedy action in $S_{t+n}$ gets us our DDQN. In the PER, our $p_t$ will be proportional to $p_t \propto ( D_{KL} (\Phi_\mathbf{z} d_t^n || d_t))^\omega $

As the Noisy Nets are trivial to adapt, we have left only the Dueling Networks. We create the shared representation $\phi = f_\xi(s)$ and then we have our probabilities of the atom $z^i$ as:

$$ p_\theta^i(s,a) = \frac{\exp(v_\eta^i(\phi) + a_\psi^i(\phi,a) - \bar{a}_\psi^i(s))}{\sum_j \exp(v_\eta^j(\phi) + a_\psi^j(\phi,a) - \bar{a}_\psi^j(s))} $$,

with $ \bar{a}_\psi^i = \frac{1}{N_{\text{actions}} } \sum_{a'} a^i_\psi (\phi,a')$,
and we are ready.



## Configurations for Colab

In [1]:
# !apt update

In [2]:
# import sys
# IN_COLAB = "google.colab" in sys.modules

# if IN_COLAB:
#     !apt install python-opengl
#     !apt install ffmpeg
#     !apt install xvfb
#     !pip install pyvirtualdisplay
#     from pyvirtualdisplay import Display
    
#     # Start virtual display
#     dis = Display(visible=0, size=(400, 400))
#     dis.start()

# 08. Rainbow

[M. Hessel et al., "Rainbow: Combining Improvements in Deep Reinforcement Learning." arXiv preprint arXiv:1710.02298, 2017.](https://arxiv.org/pdf/1710.02298.pdf)



In [ ]:
import math
import os
import random
from collections import deque
from typing import Deque, Dict, List, Tuple

import gym
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from IPython.display import clear_output
from torch.nn.utils import clip_grad_norm_

#our code
import pathlib
# download segment tree module if necessary uncomment
# if True:
#     !wget https://raw.githubusercontent.com/curt-park/rainbow-is-all-you-need/master/segment_tree.py

from segment_tree import MinSegmentTree, SumSegmentTree

## Replay buffer

Same as the basic N-step buffer. 


In [4]:
class ReplayBuffer:
    """A simple numpy replay buffer."""

    def __init__(
        self, 
        obs_dim: tuple, 
        size: int, 
        batch_size: int = 32, 
        n_step: int = 1, 
        gamma: float = 0.99
    ):
        self.obs_buf = np.zeros([size,obs_dim[0], obs_dim[1],obs_dim[2]], dtype=np.float32)
        self.next_obs_buf = np.zeros([size,obs_dim[0], obs_dim[1],obs_dim[2]], dtype=np.float32)
        self.acts_buf = np.zeros([size], dtype=np.float32)
        self.rews_buf = np.zeros([size], dtype=np.float32)
        self.done_buf = np.zeros(size, dtype=np.float32)
        self.max_size, self.batch_size = size, batch_size
        self.ptr, self.size, = 0, 0
        
        # for N-step Learning
        self.n_step_buffer = deque(maxlen=n_step)
        self.n_step = n_step
        self.gamma = gamma

    def store(
        self, 
        obs: np.ndarray, 
        act: np.ndarray, 
        rew: float, 
        next_obs: np.ndarray, 
        done: bool,
    ) -> Tuple[np.ndarray, np.ndarray, float, np.ndarray, bool]:
        transition = (obs, act, rew, next_obs, done)
        self.n_step_buffer.append(transition)

        # single step transition is not ready
        if len(self.n_step_buffer) < self.n_step:
            return ()
        
        # make a n-step transition
        rew, next_obs, done = self._get_n_step_info(
            self.n_step_buffer, self.gamma
        )
        obs, act = self.n_step_buffer[0][:2]
        
        self.obs_buf[self.ptr] = obs
        self.next_obs_buf[self.ptr] = next_obs
        self.acts_buf[self.ptr] = act
        self.rews_buf[self.ptr] = rew
        self.done_buf[self.ptr] = done
        self.ptr = (self.ptr + 1) % self.max_size
        self.size = min(self.size + 1, self.max_size)
        
        return self.n_step_buffer[0]

    def sample_batch(self) -> Dict[str, np.ndarray]:
        idxs = np.random.choice(self.size, size=self.batch_size, replace=False)

        return dict(
            obs=self.obs_buf[idxs],
            next_obs=self.next_obs_buf[idxs],
            acts=self.acts_buf[idxs],
            rews=self.rews_buf[idxs],
            done=self.done_buf[idxs],
            # for N-step Learning
            indices=indices,
        )
    
    def sample_batch_from_idxs(
        self, idxs: np.ndarray
    ) -> Dict[str, np.ndarray]:
        # for N-step Learning
        return dict(
            obs=self.obs_buf[idxs],
            next_obs=self.next_obs_buf[idxs],
            acts=self.acts_buf[idxs],
            rews=self.rews_buf[idxs],
            done=self.done_buf[idxs],
        )
    
    def _get_n_step_info(
        self, n_step_buffer: Deque, gamma: float
    ) -> Tuple[np.int64, np.ndarray, bool]:
        """Return n step rew, next_obs, and done."""
        # info of the last transition
        rew, next_obs, done = n_step_buffer[-1][-3:]

        for transition in reversed(list(n_step_buffer)[:-1]):
            r, n_o, d = transition[-3:]

            rew = r + gamma * rew * (1 - d)
            next_obs, done = (n_o, d) if d else (next_obs, done)

        return rew, next_obs, done

    def __len__(self) -> int:
        return self.size

## Prioritized replay Buffer

`store` method returns boolean in order to inform if a N-step transition has been generated.



In [5]:
class PrioritizedReplayBuffer(ReplayBuffer):
    """Prioritized Replay buffer.
    
    Attributes:
        max_priority (float): max priority
        tree_ptr (int): next index of tree
        alpha (float): alpha parameter for prioritized replay buffer
        sum_tree (SumSegmentTree): sum tree for prior
        min_tree (MinSegmentTree): min tree for min prior to get max weight
        
    """
    
    def __init__(
        self, 
        obs_dim: tuple, 
        size: int, 
        batch_size: int = 32, 
        alpha: float = 0.6,
        n_step: int = 1, 
        gamma: float = 0.99,
    ):
        """Initialization."""
        assert alpha >= 0
        
        super(PrioritizedReplayBuffer, self).__init__(
            obs_dim, size, batch_size, n_step, gamma
        )
        self.max_priority, self.tree_ptr = 1.0, 0
        self.alpha = alpha
        
        # capacity must be positive and a power of 2.
        tree_capacity = 1
        while tree_capacity < self.max_size:
            tree_capacity *= 2

        self.sum_tree = SumSegmentTree(tree_capacity)
        self.min_tree = MinSegmentTree(tree_capacity)
        
    def store(
        self, 
        obs: np.ndarray, 
        act: int, 
        rew: float, 
        next_obs: np.ndarray, 
        done: bool,
    ) -> Tuple[np.ndarray, np.ndarray, float, np.ndarray, bool]:
        """Store experience and priority."""
        transition = super().store(obs, act, rew, next_obs, done)
        
        if transition:
            self.sum_tree[self.tree_ptr] = self.max_priority ** self.alpha
            self.min_tree[self.tree_ptr] = self.max_priority ** self.alpha
            self.tree_ptr = (self.tree_ptr + 1) % self.max_size
        
        return transition

    def sample_batch(self, beta: float = 0.4) -> Dict[str, np.ndarray]:
        """Sample a batch of experiences."""
        assert len(self) >= self.batch_size
        assert beta > 0
        
        indices = self._sample_proportional()
        
        obs = self.obs_buf[indices]
        next_obs = self.next_obs_buf[indices]
        acts = self.acts_buf[indices]
        rews = self.rews_buf[indices]
        done = self.done_buf[indices]
        weights = np.array([self._calculate_weight(i, beta) for i in indices])
        
        return dict(
            obs=obs,
            next_obs=next_obs,
            acts=acts,
            rews=rews,
            done=done,
            weights=weights,
            indices=indices,
        )
        
    def update_priorities(self, indices: List[int], priorities: np.ndarray):
        """Update priorities of sampled transitions."""
        assert len(indices) == len(priorities)

        for idx, priority in zip(indices, priorities):
            assert priority > 0
            assert 0 <= idx < len(self)

            self.sum_tree[idx] = priority ** self.alpha
            self.min_tree[idx] = priority ** self.alpha

            self.max_priority = max(self.max_priority, priority)
            
    def _sample_proportional(self) -> List[int]:
        """Sample indices based on proportions."""
        indices = []
        p_total = self.sum_tree.sum(0, len(self) - 1)
        segment = p_total / self.batch_size
        
        for i in range(self.batch_size):
            a = segment * i
            b = segment * (i + 1)
            upperbound = random.uniform(a, b)
            idx = self.sum_tree.retrieve(upperbound)
            indices.append(idx)
            
        return indices
    
    def _calculate_weight(self, idx: int, beta: float):
        """Calculate the weight of the experience at idx."""
        # get max weight
        p_min = self.min_tree.min() / self.sum_tree.sum()
        max_weight = (p_min * len(self)) ** (-beta)
        
        # calculate weights
        p_sample = self.sum_tree[idx] / self.sum_tree.sum()
        weight = (p_sample * len(self)) ** (-beta)
        weight = weight / max_weight
        
        return weight

## Noisy Layer


**References:**

- https://github.com/higgsfield/RL-Adventure/blob/master/5.noisy%20dqn.ipynb
- https://github.com/Kaixhin/Rainbow/blob/master/model.py

In [6]:
class NoisyLinear(nn.Module):
    """Noisy linear module for NoisyNet.
    
    
        
    Attributes:
        in_features (int): input size of linear module
        out_features (int): output size of linear module
        std_init (float): initial std value
        weight_mu (nn.Parameter): mean value weight parameter
        weight_sigma (nn.Parameter): std value weight parameter
        bias_mu (nn.Parameter): mean value bias parameter
        bias_sigma (nn.Parameter): std value bias parameter
        
    """

    def __init__(
        self, 
        in_features: int, 
        out_features: int, 
        std_init: float = 0.5,
    ):
        """Initialization."""
        super(NoisyLinear, self).__init__()
        
        self.in_features = in_features
        self.out_features = out_features
        self.std_init = std_init

        self.weight_mu = nn.Parameter(torch.Tensor(out_features, in_features))
        self.weight_sigma = nn.Parameter(
            torch.Tensor(out_features, in_features)
        )
        self.register_buffer(
            "weight_epsilon", torch.Tensor(out_features, in_features)
        )

        self.bias_mu = nn.Parameter(torch.Tensor(out_features))
        self.bias_sigma = nn.Parameter(torch.Tensor(out_features))
        self.register_buffer("bias_epsilon", torch.Tensor(out_features))

        self.reset_parameters()
        self.reset_noise()

    def reset_parameters(self):
        """Reset trainable network parameters (factorized gaussian noise)."""
        mu_range = 1 / math.sqrt(self.in_features)
        self.weight_mu.data.uniform_(-mu_range, mu_range)
        self.weight_sigma.data.fill_(
            self.std_init / math.sqrt(self.in_features)
        )
        self.bias_mu.data.uniform_(-mu_range, mu_range)
        self.bias_sigma.data.fill_(
            self.std_init / math.sqrt(self.out_features)
        )

    def reset_noise(self):
        """Make new noise."""
        epsilon_in = self.scale_noise(self.in_features)
        epsilon_out = self.scale_noise(self.out_features)

        # outer product
        self.weight_epsilon.copy_(epsilon_out.ger(epsilon_in))
        self.bias_epsilon.copy_(epsilon_out)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Forward method implementation.
        
        We don't use separate statements on train / eval mode.
        It doesn't show remarkable difference of performance.
        """
        return F.linear(
            x,
            self.weight_mu + self.weight_sigma * self.weight_epsilon,
            self.bias_mu + self.bias_sigma * self.bias_epsilon,
        )
    
    @staticmethod
    def scale_noise(size: int) -> torch.Tensor:
        """Set scale to make noise (factorized gaussian noise)."""
        x = torch.FloatTensor(np.random.normal(loc=0.0, scale=1.0, size=size))

        return x.sign().mul(x.abs().sqrt())

## NoisyNet + DuelingNet + Categorical DQN

#### NoisyNet + DuelingNet

NoisyLinear is employed for the last two layers of advantage and value layers. The noise should be reset at evey update step.

#### DuelingNet + Categorical DQN

The dueling network architecture is adapted for use with return distributions. The network has a shared representation, which is then fed into a value stream with atom_size outputs, and into an advantage stream with atom_size × out_dim outputs. For each atom, the value and advantage streams are aggregated, as in dueling DQN, and then passed through a softmax layer to obtain the normalized parametric distributions used to estimate the returns’ distributions.

```
        advantage = self.advantage_layer(adv_hid).view(-1, self.out_dim, self.atom_size)
        value = self.value_layer(val_hid).view(-1, 1, self.atom_size)
        q_atoms = value + advantage - advantage.mean(dim=1, keepdim=True)
        
        dist = F.softmax(q_atoms, dim=-1)
```


In [7]:
#Our code
class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.size(0), -1)
################
class Network(nn.Module):
    def __init__(
        self, 
        in_dim: tuple, 
        out_dim: int, 
        atom_size: int, 
        support: torch.Tensor
    ):
        """Initialization."""
        super(Network, self).__init__()
        
        self.support = support
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.atom_size = atom_size

        # set common feature layer
        # Our code
        self.feature_layer = nn.Sequential(
            nn.Conv2d(4, 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU()
        )

        self.flatten = Flatten()
        ########################
        # set advantage layer
        self.advantage_hidden_layer = NoisyLinear(self._feature_size(), 128)
        self.advantage_layer = NoisyLinear(128, out_dim * atom_size)

        # set value layer
        self.value_hidden_layer = NoisyLinear(self._feature_size(), 128)
        self.value_layer = NoisyLinear(128, atom_size)
    #our code
    def _feature_size(self):
        return self.feature_layer(torch.zeros(1, *self.in_dim)).view(1, -1).size(1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Forward method implementation."""
        dist = self.dist(x)
        q = torch.sum(dist * self.support, dim=2)
        
        return q
    
    def dist(self, x: torch.Tensor) -> torch.Tensor:
        """Get distribution for atoms."""
        feature = self.feature_layer(x)
        fl = self.flatten(feature)
        adv_hid = F.relu(self.advantage_hidden_layer(fl))
        val_hid = F.relu(self.value_hidden_layer(fl))
        
        advantage = self.advantage_layer(adv_hid).view(
            -1, self.out_dim, self.atom_size
        )
        value = self.value_layer(val_hid).view(-1, 1, self.atom_size)
        q_atoms = value + advantage - advantage.mean(dim=1, keepdim=True)
        
        dist = F.softmax(q_atoms, dim=-1)
        dist = dist.clamp(min=1e-3)  # for avoiding nans
        
        return dist
    
    def reset_noise(self):
        """Reset all noisy layers."""
        self.advantage_hidden_layer.reset_noise()
        self.advantage_layer.reset_noise()
        self.value_hidden_layer.reset_noise()
        self.value_layer.reset_noise()

## Rainbow Agent

Here is a summary of DQNAgent class.

| Method           | Note                                                 |
| ---              | ---                                                  |
|select_action     | select an action from the input state.               |
|step              | take an action and return the response of the env.   |
|compute_dqn_loss  | return dqn loss.                                     |
|update_model      | update the model by gradient descent.                |
|target_hard_update| hard update from the local model to the target model.|
|train             | train the agent during num_frames.                   |
|test              | test the agent (1 episode).                          |
|plot              | plot the training progresses.                        |

#### Categorical DQN + Double DQN

The idea of Double Q-learning is to reduce overestimations by decomposing the max operation in the target into action selection and action evaluation. Here, we use `self.dqn` instead of `self.dqn_target` to obtain the target actions.

```
        # Categorical DQN + Double DQN
        # target_dqn is used when we don't employ double DQN
        next_action = self.dqn(next_state).argmax(1)
        next_dist = self.dqn_target.dist(next_state)
        next_dist = next_dist[range(self.batch_size), next_action]
```

In [8]:
class DQNAgent:
    """DQN Agent interacting with environment.
    
    Attribute:
        env (gym.Env): openAI Gym environment
        memory (PrioritizedReplayBuffer): replay memory to store transitions
        batch_size (int): batch size for sampling
        target_update (int): period for target model's hard update
        gamma (float): discount factor
        dqn (Network): model to train and select actions
        dqn_target (Network): target model to update
        optimizer (torch.optim): optimizer for training dqn
        transition (list): transition information including 
                           state, action, reward, next_state, done
        v_min (float): min value of support
        v_max (float): max value of support
        atom_size (int): the unit number of support
        support (torch.Tensor): support for categorical dqn
        use_n_step (bool): whether to use n_step memory
        n_step (int): step number to calculate n-step td error
        memory_n (ReplayBuffer): n-step replay buffer
    """

    def __init__(
        self, 
        env: gym.Env,
        memory_size: int,
        batch_size: int,
        target_update: int,
        gamma: float = 0.99,
        # PER parameters
        alpha: float = 0.2,
        beta: float = 0.4,
        prior_eps: float = 1e-6,
        # Categorical DQN parameters
        v_min: float = -10.0,
        v_max: float = 10.0,
        atom_size: int = 51,
        # N-step Learning
        n_step: int = 3,
    ):
        """Initialization.
        
        Args:
            env (gym.Env): openAI Gym environment
            memory_size (int): length of memory
            batch_size (int): batch size for sampling
            target_update (int): period for target model's hard update
            lr (float): learning rate
            gamma (float): discount factor
            alpha (float): determines how much prioritization is used
            beta (float): determines how much importance sampling is used
            prior_eps (float): guarantees every transition can be sampled
            v_min (float): min value of support
            v_max (float): max value of support
            atom_size (int): the unit number of support
            n_step (int): step number to calculate n-step td error
        """
        obs_dim = env.observation_space.shape
        action_dim = env.action_space.n
        
        self.env = env
        self.batch_size = batch_size
        self.target_update = target_update
        self.gamma = gamma
        
        #our code
        self.learning_start = 30000
        ###
        
        # NoisyNet: All attributes related to epsilon are removed
        
        # device: cpu / gpu
        self.device = torch.device(
            "cuda" if torch.cuda.is_available() else "cpu"
        )
        print(self.device)
        
        # PER
        # memory for 1-step Learning
        self.beta = beta
        self.prior_eps = prior_eps
        self.memory = PrioritizedReplayBuffer(
            obs_dim, memory_size, batch_size, alpha=alpha
        )
        
        # memory for N-step Learning
        self.use_n_step = True if n_step > 1 else False
        if self.use_n_step:
            self.n_step = n_step
            self.memory_n = ReplayBuffer(
                obs_dim, memory_size, batch_size, n_step=n_step, gamma=gamma
            )
            
        # Categorical DQN parameters
        self.v_min = v_min
        self.v_max = v_max
        self.atom_size = atom_size
        self.support = torch.linspace(
            self.v_min, self.v_max, self.atom_size
        ).to(self.device)

        # networks: dqn, dqn_target
        self.dqn = Network(
            obs_dim, action_dim, self.atom_size, self.support
        ).to(self.device)
        self.dqn_target = Network(
            obs_dim, action_dim, self.atom_size, self.support
        ).to(self.device)
        self.dqn_target.load_state_dict(self.dqn.state_dict())
        self.dqn_target.eval()
        
        # optimizer
        #our code
        self.optimizer = optim.Adam(self.dqn.parameters(),lr=0.0001)

        # transition to store in memory
        self.transition = list()
        #scores history
        self.scores = []
        # mode: train / test
        self.is_test = False

    def select_action(self, state: np.ndarray) -> np.ndarray:
        """Select an action from the input state."""
        # NoisyNet: no epsilon greedy action selection
        state = np.concatenate(state._frames, axis=0)
        state_exp_dims = np.expand_dims(state,axis=0)
        # print(state_exp_dims.shape)
        selected_action = self.dqn(
            torch.FloatTensor(state_exp_dims).to(self.device)
        ).argmax()
        selected_action = selected_action.detach().cpu().numpy()
        
        if not self.is_test:
            self.transition = [state, selected_action]
        
        return selected_action

    def step(self, action: np.ndarray) -> Tuple[np.ndarray, np.float64, bool]:
        """Take an action and return the response of the env."""
        next_state, reward, done, _ = self.env.step(action)
        #print(action)

        # sh =  next_state.shape
        # #next_state = np.reshape( next_state, [1,sh[0],sh[1],sh[2]])
        # next_state = np.transpose(next_state, (2,0,1))
  
        if not self.is_test:
            self.transition += [reward, next_state, done]
            
            # N-step transition
            if self.use_n_step:
                one_step_transition = self.memory_n.store(*self.transition)
            # 1-step transition
            else:
                one_step_transition = self.transition

            # add a single step transition
            if one_step_transition:
                self.memory.store(*one_step_transition)
    
        return next_state, reward, done

    def update_model(self) -> torch.Tensor:
        """Update the model by gradient descent."""
        # PER needs beta to calculate weights
        samples = self.memory.sample_batch(self.beta)
        weights = torch.FloatTensor(
            samples["weights"].reshape(-1, 1)
        ).to(self.device)
        indices = samples["indices"]
        
        # 1-step Learning loss
        elementwise_loss = self._compute_dqn_loss(samples, self.gamma)
        
        # PER: importance sampling before average
        loss = torch.mean(elementwise_loss * weights)
        
        # N-step Learning loss
        # we are gonna combine 1-step loss and n-step loss so as to
        # prevent high-variance. The original rainbow employs n-step loss only.
        if self.use_n_step:
            gamma = self.gamma ** self.n_step
            samples = self.memory_n.sample_batch_from_idxs(indices)
            elementwise_loss_n_loss = self._compute_dqn_loss(samples, gamma)
            elementwise_loss += elementwise_loss_n_loss
            # NOTE ACHAR O LR LEARNING RATE 
            # PER: importance sampling before average
            loss = torch.mean(elementwise_loss * weights)

        self.optimizer.zero_grad()
        loss.backward()
        clip_grad_norm_(self.dqn.parameters(), 10.0)
        self.optimizer.step()
        
        # PER: update priorities
        loss_for_prior = elementwise_loss.detach().cpu().numpy()
        new_priorities = loss_for_prior + self.prior_eps
        self.memory.update_priorities(indices, new_priorities)
        
        # NoisyNet: reset noise
        self.dqn.reset_noise()
        self.dqn_target.reset_noise()

        return loss.item()
        
    def train(self, num_frames: int, plotting_interval: int = 5000):
        """Train the agent."""
        self.is_test = False
        
        state = self.env.reset()

        
        # sh =  state.shape
        # #state = np.reshape( state, [1,sh[0],sh[1],sh[2]])
        
        # state = np.transpose(state, (2,0,1))

        
        update_cnt = 0
        losses = []
        self.scores = []
        strategies = []
        score = 0

        for frame_idx in range(1, num_frames + 1):
            action = self.select_action(state)
            strategies.append(action)
            next_state, reward, done = self.step(action)
            
            # print('rew = ',reward)

            state = next_state
            score += reward
            
            # NoisyNet: removed decrease of epsilon
            
            # PER: increase beta
            fraction = min(frame_idx / 100000, 1.0)
            self.beta = min(1.0,self.beta + fraction * (1.0 - self.beta))

            # if episode ends
            if done:
                state = self.env.reset()
                # sh =  state.shape
                # #state = np.reshape( state, [1,sh[0],sh[1],sh[2]])
                # state = np.transpose(state, (2,0,1))
                
                self.scores.append(score)
                score = 0

            # if training is ready
            #our code
            if len(self.memory) >= self.learning_start:
                loss = self.update_model()
                losses.append(loss)
                update_cnt += 1
                
                # if hard update is needed
                if update_cnt % self.target_update == 0:
                    self._target_hard_update()
                    self.save_model(self.dqn)

            # plotting
            if frame_idx % plotting_interval == 0:
                self._plot(frame_idx, self.scores, losses, strategies)
                
        self.save_model(self.dqn)
        self.env.close()
        
    def save_model(self, model):
        fname = "OUR_MODEL"
        
        fname = os.path.join("models", fname)

        pathlib.Path('models').mkdir(exist_ok=True)
        torch.save(model.state_dict(), fname)
                
    def load_model(self, model):
        fname = os.path.join("models", "OUR_MODEL")
        
        if args.device == torch.device("cpu"):
            map_location = lambda storage, loc: storage
        else:
            map_location = None

        if not os.path.exists(fname):
            raise ValueError("No model saved with name {}".format(fname))

        model.load_state_dict(torch.load(fname, map_location))
    
    
    def test(self) -> None:
        """Test the agent."""
        self.is_test = True
        
        state = self.env.reset()
        # sh =  state.shape
        # state = np.reshape( state, [1,sh[0],sh[1],sh[2]])
        # state = np.transpose(state, (0,3,1,2))
        done = False
        score = 0
        
        while not done:
            self.env.render()
            action = self.select_action(state)
            next_state, reward, done = self.step(action)

            state = next_state
            score += reward
        
        print("score: ", score)
        self.env.close()

    def _compute_dqn_loss(self, samples: Dict[str, np.ndarray], gamma: float) -> torch.Tensor:
        """Return categorical dqn loss."""
        device = self.device  # for shortening the following lines
        state = torch.FloatTensor(samples["obs"]).to(device)
        next_state = torch.FloatTensor(samples["next_obs"]).to(device)
        action = torch.LongTensor(samples["acts"]).to(device)
        reward = torch.FloatTensor(samples["rews"].reshape(-1, 1)).to(device)
        done = torch.FloatTensor(samples["done"].reshape(-1, 1)).to(device)
        
        # Categorical DQN algorithm
        delta_z = float(self.v_max - self.v_min) / (self.atom_size - 1)

        with torch.no_grad():
            # Double DQN
            next_action = self.dqn(next_state).argmax(1)
            next_dist = self.dqn_target.dist(next_state)
            next_dist = next_dist[range(self.batch_size), next_action]

            t_z = reward + (1 - done) * gamma * self.support
            t_z = t_z.clamp(min=self.v_min, max=self.v_max)
            b = (t_z - self.v_min) / delta_z
            l = b.floor().long()
            u = b.ceil().long()

            offset = (
                torch.linspace(
                    0, (self.batch_size - 1) * self.atom_size, self.batch_size
                ).long()
                .unsqueeze(1)
                .expand(self.batch_size, self.atom_size)
                .to(self.device)
            )

            proj_dist = torch.zeros(next_dist.size(), device=self.device)
            proj_dist.view(-1).index_add_(
                0, (l + offset).view(-1), (next_dist * (u.float() - b)).view(-1)
            )
            proj_dist.view(-1).index_add_(
                0, (u + offset).view(-1), (next_dist * (b - l.float())).view(-1)
            )

        dist = self.dqn.dist(state)
        log_p = torch.log(dist[range(self.batch_size), action])
        elementwise_loss = -(proj_dist * log_p).sum(1)

        return elementwise_loss

    def _target_hard_update(self):
        """Hard update: target <- local."""
        self.dqn_target.load_state_dict(self.dqn.state_dict())
                
    def _plot(
        self, 
        frame_idx: int, 
        scores: List[float], 
        losses: List[float],
        strategies: List[float]
    ):
        """Plot the training progresses."""
        #our code
        print('Frame idx = ',frame_idx)
        print('Max score = ',max(scores))
        print('Min score = ',min(scores))
        print('Average score = ',np.mean(scores))
        print('Average loss = ',np.mean(losses))
#         print('Last actions = ',strategies[-15:])
#         clear_output(True)
#         plt.figure(figsize=(20, 5))
#         plt.subplot(121)
#         plt.title('frame %s. score: %s' % (frame_idx, np.mean(scores[-10:])))
#         plt.plot(scores,'x')
#         plt.subplot(122)
#         plt.title('loss')
#         plt.plot(losses,'x')
#         plt.subplot(133)
#         plt.title('strategy')
#         plt.plot(strategies)
        plt.show()

## Environment Wrapper



In [9]:
""" our code - code not originally in https://github.com/Curt-Park/rainbow-is-all-you-need

Source: https://github.com/openai/baselines/blob/master/baselines/common/atari_wrappers.py
"""
import numpy as np
from collections import deque
import gym
from gym import spaces
import cv2


class NoopResetEnv(gym.Wrapper):
    def __init__(self, env=None, noop_max=30):
        """Sample initial states by taking random number of no-ops on reset.
        No-op is assumed to be action 0.
        """
        super(NoopResetEnv, self).__init__(env)
        self.noop_max = noop_max
        self.override_num_noops = None
        assert env.unwrapped.get_action_meanings()[0] == 'NOOP'

    def reset(self):
        """ Do no-op action for a number of steps in [1, noop_max]."""
        self.env.reset()
        if self.override_num_noops is not None:
            noops = self.override_num_noops
        else:
            noops = np.random.randint(1, self.noop_max + 1)
        assert noops > 0
        obs = None
        for _ in range(noops):
            obs, _, done, _ = self.env.step(0)
            if done:
                obs = self.env.reset()
        return obs


class FireResetEnv(gym.Wrapper):
    def __init__(self, env=None):
        """For environments where the user need to press FIRE for the game to start."""
        super(FireResetEnv, self).__init__(env)
        assert env.unwrapped.get_action_meanings()[1] == 'FIRE'
        assert len(env.unwrapped.get_action_meanings()) >= 3

    def reset(self):
        self.env.reset()
        obs, _, done, _ = self.env.step(1)
        if done:
            self.env.reset()
        obs, _, done, _ = self.env.step(2)
        if done:
            self.env.reset()
        return obs


class EpisodicLifeEnv(gym.Wrapper):
    def __init__(self, env=None):
        """Make end-of-life == end-of-episode, but only reset on true game over.
        Done by DeepMind for the DQN and co. since it helps value estimation.
        """
        super(EpisodicLifeEnv, self).__init__(env)
        self.lives = 0
        self.was_real_done = True
        self.was_real_reset = False

    def step(self, action):
        obs, reward, done, info = self.env.step(action)
        self.was_real_done = done
        # check current lives, make loss of life terminal,
        # then update lives to handle bonus lives
        lives = self.env.unwrapped.ale.lives()
        if lives < self.lives and lives > 0:
            # for Qbert somtimes we stay in lives == 0 condtion for a few frames
            # so its important to keep lives > 0, so that we only reset once
            # the environment advertises done.
            done = True
        self.lives = lives
        return obs, reward, done, info

    def reset(self):
        """Reset only when lives are exhausted.
        This way all states are still reachable even though lives are episodic,
        and the learner need not know about any of this behind-the-scenes.
        """
        if self.was_real_done:
            obs = self.env.reset()
            self.was_real_reset = True
        else:
            # no-op step to advance from terminal/lost life state
            obs, _, _, _ = self.env.step(0)
            self.was_real_reset = False
        self.lives = self.env.unwrapped.ale.lives()
        return obs


class ProcessFrame84(gym.ObservationWrapper):
    def __init__(self, env=None):
        super(ProcessFrame84, self).__init__(env)
        self.observation_space = spaces.Box(low=0, high=255, shape=(84, 84, 1))

    def observation(self, obs):
        return ProcessFrame84.process(obs)

    @staticmethod
    def process(frame):
        if frame.size == 210 * 160 * 3:
            img = np.reshape(frame, [210, 160, 3]).astype(np.float32)
        elif frame.size == 250 * 160 * 3:
            img = np.reshape(frame, [250, 160, 3]).astype(np.float32)
        else:
            assert False, "Unknown resolution."
        img = img[:, :, 0] * 0.299 + img[:, :, 1] * 0.587 + img[:, :, 2] * 0.114
        resized_screen = cv2.resize(img, (84, 110), interpolation=cv2.INTER_AREA)
        x_t = resized_screen[18:102, :]
        x_t = np.reshape(x_t, [84, 84, 1])
        return x_t.astype(np.uint8)


class ClippedRewardsWrapper(gym.RewardWrapper):
    def reward(self, reward):
        """Change all the positive rewards to 1, negative to -1 and keep zero."""
        return np.sign(reward)


class LazyFrames(object):
    def __init__(self, frames):
        """This object ensures that common frames between the observations are only stored once.
        It exists purely to optimize memory usage which can be huge for DQN's 1M frames replay
        buffers.
        This object should only be converted to numpy array before being passed to the model.
        You'd not belive how complex the previous solution was."""
        self._frames = frames

    def __array__(self, dtype=None):
        out = np.concatenate(self._frames, axis=0)
        if dtype is not None:
            out = out.astype(dtype)
        return out


class FrameStack(gym.Wrapper):
    def __init__(self, env, k):
        """Stack k last frames.
        Returns lazy array, which is much more memory efficient.
        See Also
        --------
        baselines.common.atari_wrappers.LazyFrames
        """
        gym.Wrapper.__init__(self, env)
        self.k = k
        self.frames = deque([], maxlen=k)
        shp = env.observation_space.shape
        self.observation_space = spaces.Box(low=0, high=255, shape=(shp[0]*k, shp[1], shp[2]))

    def reset(self):
        ob = self.env.reset()
        for _ in range(self.k):
            self.frames.append(ob)
        return self._get_ob()

    def step(self, action):
        ob, reward, done, info = self.env.step(action)
        self.frames.append(ob)
        return self._get_ob(), reward, done, info

    def _get_ob(self):
        assert len(self.frames) == self.k
        return LazyFrames(list(self.frames))


class ChannelsFirstImageShape(gym.ObservationWrapper):
    """
    Change image shape to CWH
    """
    def __init__(self, env):
        super(ChannelsFirstImageShape, self).__init__(env)
        old_shape = self.observation_space.shape
        self.observation_space = gym.spaces.Box(low=0.0, high=1.0, shape=(old_shape[-1], old_shape[0], old_shape[1]))

    def observation(self, observation):
        return np.swapaxes(observation, 2, 0)


class MainGymWrapper():

    @staticmethod
    def wrap(env):
        env = NoopResetEnv(env, noop_max=30)
        if 'FIRE' in env.unwrapped.get_action_meanings():
            env = FireResetEnv(env)
        env = ProcessFrame84(env)
        env = ChannelsFirstImageShape(env)
        env = FrameStack(env, 4)
        # env = ClippedRewardsWrapper(env)
        return env

In [10]:
# environment initialization, our code
env_id = "PongNoFrameskip-v4"
env = MainGymWrapper.wrap(gym.make(env_id))

/home/l171855/anaconda3/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [11]:
env.observation_space.shape

(4, 84, 84)

## Set random seed

In [12]:
seed = 1122

def seed_torch(seed):
    torch.manual_seed(seed)
    if torch.backends.cudnn.enabled:
        torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.deterministic = True

np.random.seed(seed)
random.seed(seed)
seed_torch(seed)
env.seed(seed)

[1122, 1711756444]

## Initialize

In [13]:
# parameters
# num_frames = 100000
#our code our parameter tuning
num_frames = 1400000
memory_size = int(1e5)
batch_size = 32
target_update = 10000
gamma = 0.99
alpha = 0.6

# train
agent = DQNAgent(env, memory_size, batch_size, target_update, gamma = gamma, alpha = alpha)

cuda


## Train

In [ ]:
agent.train(num_frames)

Frame idx =  5000
Max score =  -21.0
Min score =  -21.0
Average score =  -21.0
Average loss =  nan


/home/l171855/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/l171855/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Frame idx =  10000
Max score =  -21.0
Min score =  -21.0
Average score =  -21.0
Average loss =  nan
Frame idx =  15000
Max score =  -21.0
Min score =  -21.0
Average score =  -21.0
Average loss =  nan
Frame idx =  20000
Max score =  -21.0
Min score =  -21.0
Average score =  -21.0
Average loss =  nan
Frame idx =  25000
Max score =  -21.0
Min score =  -21.0
Average score =  -21.0
Average loss =  nan
Frame idx =  30000
Max score =  -21.0
Min score =  -21.0
Average score =  -21.0
Average loss =  nan



## Test


In [ ]:
# agent.env = gym.wrappers.Monitor(env, "videos", force=True)
# agent.test()

## Render

In [ ]:
# import base64
# import glob
# # import io
# import os

# from IPython.display import HTML, display


# def ipython_show_video(path: str) -> None:
#     """Show a video at `path` within IPython Notebook."""
#     if not os.path.isfile(path):
#         raise NameError("Cannot access: {}".format(path))

#     video = io.open(path, "r+b").read()
#     encoded = base64.b64encode(video)

#     display(HTML(
#         data="""
#         <video alt="test" controls>
#         <source src="data:video/mp4;base64,{0}" type="video/mp4" />
#         </video>
#         """.format(encoded.decode("ascii"))
#     ))

# list_of_files = glob.glob("videos/*.mp4")
# latest_file = max(list_of_files, key=os.path.getctime)
# print(latest_file)
# ipython_show_video(latest_file)